In [3]:
import pandas as pd
import numpy as np

# Correlation vs Causation
Simpsons Paradox in the number of students entering majors in UC Berkeley
Another example of lack of insight, is the confusion of correlation and causation

Let's see an example where we may be tempted to reach the wrong conclusion

*I am reminded* that we discussed the "Admissions Case Study" in Lesson 7, for which I didn't make a note?
https://classroom.udacity.com/courses/st101/lessons/48759015/concepts/487550470923

Oh yes I do have notes, they're just in the first section 
http://localhost:8888/notebooks/udacity_notes/st101__1_Visualizing_relationships_in_data__7_Admissions%20Case%20Study.ipynb

I am reminded of the data for Simpsons paradox. In each of their Major, females  had higher rates of admission as a proportion of female applicants. But in total, (i.e. across all majors; ..sum() the panel...) , women were accepted at a lower rate. 


In [8]:
def admissions_3_panel():
    return pd.Panel({'Male': pd.DataFrame({
                    'Applied': pd.Series({'Major A': 900, 'Major B':100}), 
                    'Admitted': pd.Series({'Major A' : 450, 'Major B': 10})
    }),
                 'Female': pd.DataFrame({
                     'Applied': pd.Series({'Major A' : 100, 'Major B': 900}),
                     'Admitted': pd.Series({ 'Major A': 80, 'Major B': 180})
                 })})
    return panel
admissions_3_panel().sum()

,Female,Male
Admitted,260,460
Applied,1000,1000


In [11]:
#There is no need to compare the denominator; the number of those who applied
#in each gender, since those numbers are the same.
#Lets just consider the numerator; look how many more men were admitted than women;
admissions_3_panel().sum().loc['Admitted']
# No need to consider denom / admissions_3_panel().sum().loc['Applied']

Female    260
Male      460
Name: Admitted, dtype: int64

So how can we explain this? 
 
1. Women were "smarter" about where they applied individually? Or however to characterize: avoiding "high rates of rejection" that might seem typical of a Major, given men's rates?
2. The only reason Women were considered to be favored, is that they had higer rates a) in a low-rate-(< 50%) value case; (i.e. Major B: 10% men vs 20% women), where the numbe of women applied was very *high*; 900, and generally men applied very *low*. "Beating" an admission proportion for very few men is not very relevant. On the other hand, seeing as only 180 women were admitted; the *low-rate* effect on a large number, the .sum() doesn't do much for women in total
3. "If more women would apply to Major A instead of Major B, we wouldnt observe any difference, or we may see a *total* bias towards women as well!"

In the above case; we needed to *aggregate* (i.e. sum up *across* Majors) to see the true effect of lower-admission-rates-for-women.

In this Correlation vs Causation lesson we look at a similar phenomenon , but it goes the other way; we gain insight by *subdividing* our population into groups: those who are sick and those who are healthy ( a much more likely cause of death); keep this in mind for when Sebastian distinguishes between 'Sick' and 'Healthy'

In my above list, trying to understand Simpsons paradox, I suggested "If theyd just pick one Major over another". I wanted a real example so I could rationalize why someone might *prefer* one over another. I could only wonder if one Major were *more difficult* than the other. I can't be sure? (and even if I were, I don't want to speculate about a gender preference for *more difficult things* ?), but I realize *aggregating on a different axis* is another way of *ignoring* gender, to see simply admission rates in each Major. Major B has lower admission rate; you're ~~more than twice as likely~~ almost three times as likely (2.78) to get admitted to Major A than Major B.

In [13]:
admissions_3_panel().sum(axis=0)

,Admitted,Applied
Major A,530,1000
Major B,190,1000


In [14]:
admitted=admissions_3_panel().sum(axis=0)['Admitted']
admitted['Major A'] / admitted['Major B']

2.7894736842105261

In [23]:
hospital_vs_home = pd.DataFrame(index=('total', 'died'), data={'in hospital': [40, 4], 'at home': [8000, 20]}, columns=['in hospital', 'at home'])

In [24]:
hospital_vs_home

,in hospital,at home
total,40,8000
died,4,20


# Indexing in Pandas

I keep getting confused on this, hopefully this helps me remember what to avoid:
## Don't use `ix`, use `loc` or `iloc`
## Don't use `(...)`, use `[ indexer ]`

In [25]:
hospital_vs_home.loc['died']/ hospital_vs_home.loc['total']

in hospital    0.1000
at home        0.0025
dtype: float64

In [36]:

def quiz_mortality():
    """Compute for me the percentages of the people who died -- in the hospital, and at home"""
    return (hospital_vs_home.loc['died']/ hospital_vs_home.loc['total'])

In [38]:
actual_values = quiz_mortality().values
expected_values =  [0.1000, 0.0025]
assertion_msg = 'Expect the [hospital death rate, at-home death rate] to be {} , but actual values were {}'.format(expected_values, actual_values)
assert np.allclose(actual_values, expected_values), assertion_msg


# Chances
Chances of dying in a hospital are 40 times larger than the chances of dying at home?
This is an illustration of correlation
## Correlation
See next example; 40 times larger:

In [39]:
quiz_mortality()['in hospital'] / quiz_mortality()['at home']

40.0

In [40]:
def quiz_deciding():
    """As a smart statistics student, knowing your chances of dying in the hospital are higher, will you stay at home"""
    #I cannot explain why I should answer False, except Sebastian seems to be setting us up to expect that:
    return False

# Difference in  Language: Correlation vs Causation
## Correlation Statement

 > Chances of dying in a hospital are 40 times larger than at home

## Causation statement

 > Being in a hospital --and being in a hospital alone, i.e. no other cause -- increaes your probability of dying by a factor of 40

I'll admit I have a hard time understanding the semantic difference; can we get more precise language to describe it? Notice I added some hyphens around the part that is implied in "Causation"; Sebastian whispers it


In [15]:
sick_healthy_panel = pd.Panel({'in hospital': {
    'total' : {
        'sick': 36,
        'healthy':4
    },
    'died': {
        'sick':4,
        'healthy':0
    }
},
          'at home': {
              'total' : {
                  'sick': 40,
                  'healthy': 7960
              },
              'died' : {
                  'sick': 20,
                  'healthy': 20
              }
          }
         })

In [21]:
sick_healthy_panel

<class 'pandas.core.panel.Panel'>
Dimensions: 2 (items) x 2 (major_axis) x 2 (minor_axis)
Items axis: at home to in hospital
Major_axis axis: healthy to sick
Minor_axis axis: died to total

In [22]:
sick_healthy_panel.sum()

,at home,in hospital
died,40,4
total,8000,40


This not the dataframe I expected; it erased the "subdivision" between sick and healthy! Let me try using pd.Series and pd.DataFrame:

In [19]:
sick_healthy_panel_df = pd.Panel({'in hospital': pd.DataFrame({
    'total' : pd.Series({
        'sick': 36,
        'healthy':4
    }),
    'died': pd.Series({
        'sick':4,
        'healthy':0
    })
}),
          'at home': pd.DataFrame({
              'total' : pd.Series({
                  'sick': 40,
                  'healthy': 7960
              }),
              'died' : pd.Series({
                  'sick': 20,
                  'healthy': 20
              })
          })
         })

In [20]:
sick_healthy_panel_df.sum()

,at home,in hospital
died,40,4
total,8000,40


Still no! So, lesson

1. The highest level that you pass to panel, i.e. the first level of keys remains the "lowest level" how I think of the data structure; that is,  ~~the index of the Series~~ not true! the column header names! But columns are  the **last** level to be summed... a bit counterintuitive for me...
2. The lowest level therefore, is the highest data structure level; the "frames" of the panel...
3. The "frames" of the panel are the **first** to be summed when doing sick_healthy_panel.sum() and since were interested in distinguishing sick vs healthy, we gotta rearrange!
4. The way Sebastians written it, it's hard to contort into sick first as the highest level (they are separate rows in separate tables), but the middle row is OK, I can always Transpose a DataFrame, can I pivot the panel too?...


5. No need to use pd.DataFrame or pd.Series except that it helps me be *sure/explicit* of what the values will come to represent; just remember the "top-to-bottom, bottom-to-top" discussed above

In [24]:
sick_healthy_panel_p = pd.Panel({'in hospital': {
    'sick' : {
        'total': 36,
        'died':4
    },
    'healthy': {
        'total':4,
        'died':0
    }
},
          'at home': {
              'sick' : {
                  'total': 40,
                  'died': 20
              },
              'healthy' : {
                  'total': 7960,
                  'died': 20
              }
          }
         })

In [25]:
sick_healthy_panel_p.sum()

,at home,in hospital
healthy,7980,4
sick,60,40


In [26]:
sick_healthy_panel_p.sum().T

,healthy,sick
at home,7980,60
in hospital,4,40


# Ugh! Wrong again...
I dont want to *add together* total + died; that distinction must be preserved to the last sum()! 
So I guess I'll bite the bullet and arrange these in order from most important to least important, which are, in order:
 1. total vs died
 2. sick vs healthy
 3. *then* hospital vs home (which we consider a confounder, or whatever its called)

 --- 
 (notice, the first two items alone are sufficient; two dimension: a dataframe is gives us the  different "view" on the data which suits our new insight/guess about the cause)

In [27]:
sick_healthy_panel_f = pd.Panel({'total': {
    'sick' : {
        'in hospital': 36,
        'at home':40
    },
    'healthy': {
        'in hospital':4,
        'at home':7960
    }
},
          'died': {
              'sick' : {
                  'in hospital': 4,
                  'at home': 20
              },
              'healthy' : {
                  'in hospital': 0,
                  'at home': 20
              }
          }
         })

In [32]:
#Oops, but died and total become the columns...
sick_healthy_panel_f.sum()
#It does *not* become what I thought it would;
#I thought.... died and total would be Series index; changed my verbiage above
#However, sum() sums across Series index before it Sums across Columns, so
#sum also doesnt behave as I expect... meaning the sum().sum() still
#preserves died/total distinction to the end:

,died,total
healthy,20,7964
sick,24,76


In [33]:

sick_healthy_panel_f.sum().sum()

died       44
total    8040
dtype: int64

In [49]:
from IPython.core.display import display, HTML


In [50]:
def considering_health():
    """
    Please now fill out in percent, what is the percentage
    of people that passed away, in each of the four columns
    Using the order-of-indexing, which actually accept indexers 
    over your *Items* axis: Items axis: died to total, *first*
    Even though those are the *last* to be summed.
    Then uses a neat stack trick to print them in order Sebastian
    asks for them
    https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.stack.html
    Adn then formats them properly, how nice!
    https://stackoverflow.com/a/36175424/1175496
    """
    fractions = sick_healthy_panel_f['died']/sick_healthy_panel_f['total']
    
    stack = fractions.stack()
    
    #Slight tweak using the `float_format` arg instead of `formatters` arg
    html = stack.to_html(float_format='{:,.2%}'.format)
    
    display(HTML(html))
    
    

In [51]:
considering_health()

AttributeError: 'Series' object has no attribute 'to_html'

In [42]:
sick_healthy_panel_f

<class 'pandas.core.panel.Panel'>
Dimensions: 2 (items) x 2 (major_axis) x 2 (minor_axis)
Items axis: died to total
Major_axis axis: at home to in hospital
Minor_axis axis: healthy to sick

In [48]:
sick_healthy_panel_f.sum()

,died,total
healthy,20,7964
sick,24,76


In [43]:
#So thats the effect for summing, which is nice
#And then, for convenience, indexing goes in order of the dictionary depth
#Which, I realize, is called:
#Items
#Major
#Minor
#Which, remember, the Major is summed *first* !
#*then* Minor
#*then* Items
sick_healthy_panel_f
sick_healthy_panel_f['died',:,:]

,healthy,sick
at home,20,20
in hospital,0,4


In [38]:
 sick_healthy_panel_f.loc[:,:,'died']

KeyError: 'the label [died] is not in the [minor_axis]'